# From Aggression to Tradition: The Evolution of Fight Song Tropes

College fight songs are among the oldest and most persistent traditions in American sports. Thousands of fans still sing anthems that are over a century old. But do these songs reflect modern values? This project explores how the "DNA" of a fight song has shifted from rousing calls to fight and win in early songs to school-spirited anthems celebrating tradition in modern eras.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

path = '/content/drive/MyDrive/Dataset_LoveDataWeek/fight-songs-updated.csv'
df = pd.read_csv(path)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

df['year_numeric'] = pd.to_numeric(df['year'], errors='coerce')

def assign_era(year):
  if pd.isna(year):
    return 'Unknown'
  year = int(year)
  if year < 1925:
    return 'Pre-1925'
  if year < 1966:
    return '1925–1965'
  if year < 2000:
    return '1966–1999'
  return '2000+'

df['era'] = df['year_numeric'].apply(assign_era)

tropes = ['fight', 'victory', 'win_won', 'rah', 'nonsense', 'colors', 'men', 'spelling']

df_tropes = df.copy()
for col in tropes:
  df_tropes[col] = df_tropes[col].map({'Yes': 1, 'No': 0}).fillna(0).astype(int)

era_order = ['Pre-1925', '1925–1965', '1966–1999', '2000+', 'Unknown']

trope_order = (df_tropes[tropes].mean().sort_values(ascending=False).index.tolist())


# Radar plot: trope evaluation

era_means = df_tropes.groupby('era')[tropes].mean().reindex(era_order).fillna(0)

overall_mean = df_tropes[tropes].mean().tolist()

colors = px.colors.qualitative.Safe

fig_radar = go.Figure()

for i, era in enumerate(era_order):
  if era_means.loc[era].sum() > 0:
    fig_radar.add_trace(go.Scatterpolar(
        r=era_means.loc[era].tolist(),
        theta=trope_order,
        fill='toself',
        name=era,
        line_color=colors[i % len(colors)],
        opacity=0.6,
        hovertemplate="%{theta}: %{r:.1%}<extra></extra>"
    ))

fig_radar.add_trace(go.Scatterpolar(
    r=overall_mean,
    theta=trope_order,
    name='Overall Average',
    line=dict(color='white', dash='dash', width=1),
    opacity=0.5,
    hovertemplate="%{theta}: %{r:.1%}<extra></extra>"
))

fig_radar.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1], tickformat=".0%")),
    title="Proportion of common lyrical tropes by era",
    template="plotly_dark",
    legend_title="Era"
)


# Heatmap: era × trope × conference

trope_era_conf = (df_tropes.groupby(['conference', 'era'])[trope_order].mean().reset_index())

long = trope_era_conf.melt(
    id_vars=['conference', 'era'],
    value_vars=trope_order,
    var_name='trope',
    value_name='proportion'
)

fig_heatmap = px.density_heatmap(
    long,
    x='era',
    y='trope',
    z='proportion',
    facet_col='conference',
    color_continuous_scale='Viridis',
    category_orders={'era': era_order, 'trope': trope_order},
    title="How Fight Song Language Evolved Across Eras and Conferences"
)

fig_heatmap.update_traces(
    hovertemplate=(
        "Conference: %{facet_col}<br>"
        "Era: %{x}<br>"
        "Trope: %{y}<br>"
        "Proportion: %{z:.1%}<extra></extra>"
    )
)

fig_heatmap.update_layout(
    height=650,
    coloraxis_colorbar=dict(title="Proportion"),
    template='plotly_dark',
    margin=dict(l=80, r=40, t=80, b=40)
)

fig_radar.show()
fig_heatmap.show()

According to the radar plot, early songs are dominated by aggressive tropes like "fight", "victory", and references to men, while modern songs show a flatter distribution, which indicates a shift toward school identity and tradition.

The heatmap adds another layer by showing how these shifts vary across conferences. Some conferences retain traditional tropes longer than others, highlighting regional differences in how institutions balance tradition with evolving cultural norms.

Together, these visualizations show that even century-old songs adapt over time while still uniting fans, illustrating the subtle evolution of cultural expression in college sports.